In [1]:
from bs4 import BeautifulSoup as bs
import requests

#need to look at terms and conditions to know if you can scrape the website
#most websites have a file called robots.txt that you can look at to see
#if scraping is allowed. ex: google.com/robots.txt
url = 'https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO'

#I never could get dndbeyond to work for me, so I looked for a similar website and dnd.wizards seemed pretty
#close so I used that instead and it had no issues.

In [3]:
input_type = 'restaurants'
input_location = 'Manchester%2C+MO'
url_build = 'https://www.yelp.com/search?' + 'cflt=' + input_type + '&find_loc=' + input_location
print(url_build)

#Note this is all contained in my form for the python app

https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO


In [4]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome',**executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
There is no [mac64] chromedriver for browser 92.0.4515 in cache
Get LATEST driver version for 92.0.4515
Trying to download new driver from https://chromedriver.storage.googleapis.com/92.0.4515.107/chromedriver_mac64.zip
Driver has been saved in cache [/Users/jenniferarbuszewski/.wdm/drivers/chromedriver/mac64/92.0.4515.107]


In [10]:
x = browser.visit(url_build)
soup = bs(browser.html, 'html.parser')
#print(browser.html)
all_divs = soup.find_all('h4',class_='css-1l5lt1i')
print(all_divs[0])

<h4 class="css-1l5lt1i"><span class="css-1pxmz4g"><a class="css-166la90" href="/adredir?ad_business_id=cVZ4whFwPNA721C4DIetbg&amp;campaign_id=AXcQ5HkcqxT-QWqHFf0FZA&amp;click_origin=search_results&amp;placement=above_search&amp;placement_slot=0&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Ffirst-watch-des-peres-3&amp;request_id=c10f8095202a6aea&amp;signature=c7f4d7d6f12dbde24b42e4cfabf5164c0a5151efccdd68c083d378e4094ca3da&amp;slot=0" name="First Watch" rel="" target="">First Watch</a></span></h4>


In [11]:
all_divs = soup.find_all('h4',class_='css-1l5lt1i')
print(all_divs[0])

<h4 class="css-1l5lt1i"><span class="css-1pxmz4g"><a class="css-166la90" href="/adredir?ad_business_id=cVZ4whFwPNA721C4DIetbg&amp;campaign_id=AXcQ5HkcqxT-QWqHFf0FZA&amp;click_origin=search_results&amp;placement=above_search&amp;placement_slot=0&amp;redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Ffirst-watch-des-peres-3&amp;request_id=c10f8095202a6aea&amp;signature=c7f4d7d6f12dbde24b42e4cfabf5164c0a5151efccdd68c083d378e4094ca3da&amp;slot=0" name="First Watch" rel="" target="">First Watch</a></span></h4>


In [12]:
print(all_divs[2])

<h4 class="css-1l5lt1i"><span class="css-1pxmz4g">1<!-- -->. <a class="css-166la90" href="/biz/the-tavern-kitchen-and-bar-saint-louis-2" name="The Tavern Kitchen &amp; Bar" rel="" target="">The Tavern Kitchen &amp; Bar</a></span></h4>


# Pulling addressing and making sure the code works. NOTE: this worked absolutely fine until I got BANNED from Yelp!!!

In [120]:
name_url_list = []

for div in all_divs:
    try:
        name_url_element = div.find('a', href=True)
        name_url = name_url_element['href']
        if not name_url.startswith('/adredir'):
            name_url_list.append(name_url)
            print(name_url_element['name'])
    except AttributeError as e:
        print(e)

#for a in all_divs.find_all('a', href=True):
  #  name_url = a['href']

print(name_url_list)
print(len(name_url_list))

address_list = []
street_address_list = []
city_address_list = []

for site in name_url_list:
    #print(site)

    url2 = 'https://yelp.com' + site
    #browser.visit(url2)
    #soup = bs(browser.html, 'html.parser')
    #print(soup)
    response = requests.get(url2)
    #print(response)
    soup = bs(response.text, 'html.parser')
    
    addresses = soup.find('address')
    if addresses is None:
        print(site)
        print(soup)
    all_addresses = addresses.find_all('p')
    

    #for a in all_addresses:
       # restaurant_address = a.find('span', class_='raw__373c0__3-nUb')
    street_address = all_addresses[0].text
    city_address = all_addresses[1].text
    address_dict = {'street': street_address, 'city': city_address}
    address_list.append(address_dict)
    street_address_list.append(street_address)
print(address_list)

    
  
    
#<address class=""><a href="/map/the-tavern-kitchen-and-bar-saint-louis-2" class="css-ac8spe" target="" name="" rel="" role="link"><p class=" css-1bmgof7"><span class=" raw__373c0__3-nUb">2961 Dougherty Ferry Rd</span></p></a><p class=" css-n6i4z7"><span class=" raw__373c0__3-nUb">Saint Louis, MO 63122</span></p></address>

The Tavern Kitchen & Bar
The Port of Peri Peri - Ballwin
1356 Public House
Rozanna
Joey B’s Food & Drink
Malinche Mexican Culinary Experience
Ramen Tei
Circle 7 Ranch
Mia Sorella
The Wolf
['/biz/the-tavern-kitchen-and-bar-saint-louis-2', '/biz/the-port-of-peri-peri-ballwin-ballwin', '/biz/1356-public-house-twin-oaks', '/biz/rozanna-manchester', '/biz/joey-bs-food-and-drink-ballwin', '/biz/malinche-mexican-culinary-experience-ellisville', '/biz/ramen-tei-st-louis-county', '/biz/circle-7-ranch-ballwin', '/biz/mia-sorella-ballwin-3', '/biz/the-wolf-ballwin']
10
/biz/the-tavern-kitchen-and-bar-saint-louis-2
<!DOCTYPE html>

<!--[if lt IE 7 ]> <html class="ie6 ie ltie9 ltie8 no-js" lang="en-US"> <![endif]-->
<!--[if IE 7 ]>    <html class="ie7 ie ltie9 ltie8 no-js" lang="en-US"> <![endif]-->
<!--[if IE 8 ]>    <html class="ie8 ie ltie9 no-js" lang="en-US"> <![endif]-->
<!--[if IE 9 ]>    <html class="ie9 ie no-js" lang="en-US"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!--> <html class="no-js"

AttributeError: 'NoneType' object has no attribute 'find_all'

In [118]:
#verifying addresses worked

print(all_addresses)

[<p class="css-1bmgof7"><span class="raw__373c0__3-nUb">14445 Manchester Rd</span></p>, <p class="css-n6i4z7"><span class="raw__373c0__3-nUb">Ballwin, MO 63011</span></p>]


# Getting the rest of the function to work (Note: some of this comes from my form in the Python App)

In [81]:
import pandas as pd
all_restaurants = []
name_url_list = []
name_list = []
num_review_list = []
food_type_list = []
restaurant_type_list = []
star_rating_list = []
price_cat_list = []
address_list = []
    
for i in range(0,5):
    input_type = 'restaurants'
    input_location = 'Manchester%2C+MO'
    if i == 0:
        url_build = f'https://www.yelp.com/search?cflt={input_type}&find_loc={input_location}'
    else:
        url_build = f'https://www.yelp.com/search?cflt={input_type}&find_loc={input_location}&start={i*10}' 

   # url = f'url_build{i}'
    response = requests.get(url_build)
    #print(response)
    soup = bs(response.text, 'html.parser')
   # all_divs = soup.find_all('div',class_='quote')

    all_info = soup.find_all('div',class_='arrange__09f24__1Vghi border-color--default__09f24__3Epto')
    #print('length:', len(all_info))
    #print(all_info[0])

 

    for i in all_info:
        try:
            #To find the name of each restaurant
            info_element = i.find('h4',class_='css-1l5lt1i')
            if info_element is None:
                continue
            name_url_element = info_element.find('a', href=True)
            name_url = name_url_element['href']
            if name_url.startswith('/adredir'):
                continue
            #name_url_list.append(name_url)
            #name_list.append(name_url_element['name'])
            name=name_url_element['name']
        
            #To find the number of reviews for each restaurant
            num_review_element = i.find('span', class_='reviewCount__09f24__3GsGY css-e81eai')
            #num_review_list.append(num_review_element.text)
            num_review = num_review_element.text
        
            #To find the average rating for each restaurant
            star_rating_element = i.find('div', class_='i-stars__09f24___sZu0')
            #star_rating_list.append(star_rating_element['aria-label'])
            star_rating = star_rating_element['aria-label']
        
            #To find price category tag
            price_cat_element = i.find('span', class_='priceRange__09f24__2GspP css-e81eai')
            if price_cat_element is None:
                #price_cat_list.append('none_given')
                price_cat = 'undefined'
            else:
                #price_cat_list.append(price_cat_element.text)
                price_cat = price_cat_element.text
            
                
            rest_dict = {'restaurant_name': name,
                        'restaurant_rating': star_rating,
                        'restaurant_number_reviews': num_review,
                        'restaurant_price_range':price_cat }
   
            all_restaurants.append(rest_dict)
    
        except AttributeError as e:
            print(e)
            print(info_element)
            
    

    
    
       # rest_dict = {'restaurant_name': name_list[1],
                      #  'restaurant_rating': star_rating_list[1],
                      #  'restaurant_number_reviews': num_review_list[1],
                      #  'restaurant_price_range':price_cat_list[1] }

df = pd.DataFrame(all_restaurants)


    #for div in all_divs:
       # try:
           # quote_text = div.find('span',class_='text').text
            #author = div.find('small', class_='author').text
            #tags = div.find('div',class_='tags').find_all('a')
            #tags_list=[]
            #for tag in tags:
                #tags_list.append(tag.text)
            #quote_dict = {'quote':quote_text,
                        # 'author':author,
                        # 'tags':tags_list}
            
            
            #all_quotes.append(quote_dict)
        #except AttributeError as e:
            #print(e)
            
#df = pd.DataFrame(all_restaurants)
df

,restaurant_name,restaurant_rating,restaurant_number_reviews,restaurant_price_range
0,The Tavern Kitchen & Bar,4.5 star rating,355,$$$
1,Rozanna,4.5 star rating,36,undefined
2,The Port of Peri Peri - Ballwin,4 star rating,7,undefined
3,1356 Public House,4 star rating,58,$$
4,Malinche Mexican Culinary Experience,5 star rating,144,$$
5,The Wolf,4.5 star rating,295,$$
6,Ramen Tei,4 star rating,152,$$
7,Joey B’s Food & Drink,3.5 star rating,271,$$
8,Shack Breakfast & Lunch,4.5 star rating,220,$$
9,El Toluco Taqueria & Grocery,4.5 star rating,140,$


# Making a function to handle all fields from above (combining details and addresses)

In [107]:


def yelp_scraper(new_info):
    import pandas as pd
    all_restaurants = []
            new_info = {"type": new_type, "location": new_location, 
            "get_address_details": get_address_details}
    user_input_type = new_info["type"]
    user_input_location = new_info["location"]
    get_address_details = new_info["get_address_details"]

    for i in range(0,5):

        if i == 0:
            url_build = f'https://www.yelp.com/search?cflt={user_input_type}&find_loc={user_input_location}'
        else:
            url_build = f'https://www.yelp.com/search?cflt={user_input_type}&find_loc={user_input_location}&start={i*10}' 


        response = requests.get(url_build)

        soup = bs(response.text, 'html.parser')

        sleep(5)
        
        all_info = soup.find_all('div',class_='arrange__09f24__1Vghi border-color--default__09f24__3Epto')


 

        for i in all_info:
            try:
                #To find the name of each restaurant
                info_element = i.find('h4',class_='css-1l5lt1i')
                if info_element is None:
                    continue
                name_url_element = info_element.find('a', href=True)
                name_url = name_url_element['href']
                if name_url.startswith('/adredir'):
                    continue
                name=name_url_element['name']
        
                #To find the number of reviews for each restaurant
                num_review_element = i.find('span', class_='reviewCount__09f24__3GsGY css-e81eai')
                num_review = num_review_element.text
        
                #To find the average rating for each restaurant
                star_rating_element = i.find('div', class_='i-stars__09f24___sZu0')
                star_rating = star_rating_element['aria-label']
        
                #To find price category tag
                price_cat_element = i.find('span', class_='priceRange__09f24__2GspP css-e81eai')
                if price_cat_element is None:
                    price_cat = 'undefined'
                else:
                    price_cat = price_cat_element.text
            
            
                #Couldn't test because Yelp IP banned me (but it was working on it's own prior to ban)
                
                #print(site)
                if get_address_details == 'yes':
                    
                    url2 = 'https://yelp.com' + name_url
                    #browser.visit(url2)
                    #soup = bs(browser.html, 'html.parser')
                    #print(soup)
                    response = requests.get(url2)
                    #print(response)
                    soup = bs(response.text, 'html.parser')
                    
                    sleep(5)
    
                    addresses = soup.find('address')
                    if addresses is None:
                        print(site)
                        print(soup)
                    all_addresses = addresses.find_all('p')
    

                    #for a in all_addresses:
                    # restaurant_address = a.find('span', class_='raw__373c0__3-nUb')
                    street_address = all_addresses[0].text
                    city_address = all_addresses[1].text
                    #address_dict = {'street': street_address, 'city': city_address}
                    #address_list.append(address_dict)
                    #street_address_list.append(street_address)
                
                else:
                    street_address = 'did not fetch'
                    city_address = 'did not fetch'
                #End Yelp nonsense
                
                rest_dict = {'restaurant_name': name,
                            'restaurant_rating': star_rating,
                            'restaurant_number_reviews': num_review,
                            'restaurant_price_range':price_cat,
                            'street_address': street_address,
                            'city_address': city_address}
   
                all_restaurants.append(rest_dict)
    
            except AttributeError as e:
                print(e)
                #print(info_element)
    return all_restaurants
                
input_type = 'restaurants'
input_location = 'Manchester%2C+MO'

yelp_scraper(input_type, input_location)

# ********* End of Working Function*********

# Below are just notes and tests to myself

In [78]:
for i in range(0,5):
    input_type = 'restaurants'
    input_location = 'Manchester%2C+MO'
    if i == 0:
        url_build = 'https://www.yelp.com/search?' + 'cflt=' + input_type + '&find_loc=' + input_location
    else:
        url_build = f'https://www.yelp.com/search?cflt={input_type}&find_loc={input_location}&start={i*10}' 
    #f'https://quotes.toscrape.com/page/{i}'
    print(url_build)

https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO
https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO&start=10
https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO&start=20
https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO&start=30
https://www.yelp.com/search?cflt=restaurants&find_loc=Manchester%2C+MO&start=40


In [53]:
url_build_test = 'https://www.yelp.com/search?' + 'cflt=' + input_type + '&find_loc=' + input_location + '&start=' + 'val' 

In [47]:

    name_url_list = []
    name_list = []
    num_review_list = []
    food_type_list = []
    restaurant_type_list = []
    star_rating_list = []
    price_cat_list = []

    for i in all_info:
        try:
            #To find the name of each restaurant
            info_element = i.find('h4',class_='css-1l5lt1i')
            if info_element is None:
                continue
            name_url_element = info_element.find('a', href=True)
            name_url = name_url_element['href']
            if name_url.startswith('/adredir'):
                continue
            name_url_list.append(name_url)
            name_list.append(name_url_element['name'])
        
            #To find the number of reviews for each restaurant
            num_review_element = i.find('span', class_='reviewCount__09f24__3GsGY css-e81eai')
            num_review_list.append(num_review_element.text)
        
            #To find the average rating for each restaurant
            star_rating_element = i.find('div', class_='i-stars__09f24___sZu0')
            star_rating_list.append(star_rating_element['aria-label'])
        
            #To find price category tag
            price_cat_element = i.find('span', class_='priceRange__09f24__2GspP css-e81eai')
            if price_cat_element is None:
                price_cat_list.append('none_given')
            else:
                price_cat_list.append(price_cat_element.text)
       
        
        except AttributeError as e:
            print(e)
            print(info_element)
            #name_url = name_url_element['href']
            #if not name_url.startswith('/adredir'):
                #name_url_list.append(name_url)
                #print(name_url_element['name'])


#for a in all_divs.find_all('a', href=True):
  #  name_url = a['href']

#print(name_url_list)
#print(len(name_url_list))

['$$$']
['$$$', 'none_given']
['$$$', 'none_given', '$$']
['$$$', 'none_given', '$$', 'none_given']
['$$$', 'none_given', '$$', 'none_given', '$$']
['$$$', 'none_given', '$$', 'none_given', '$$', '$$']
['$$$', 'none_given', '$$', 'none_given', '$$', '$$', '$$']
['$$$', 'none_given', '$$', 'none_given', '$$', '$$', '$$', '$$']
['$$$', 'none_given', '$$', 'none_given', '$$', '$$', '$$', '$$', '$$']
['$$$', 'none_given', '$$', 'none_given', '$$', '$$', '$$', '$$', '$$', '$$']


In [ ]:
review count: 
    <span class="reviewCount__09f24__3GsGY css-e81eai">270</span>
    
average review:
    <div class=" i-stars__09f24___sZu0 i-stars--regular-3-half__09f24__1ufzF border-color--default__09f24__3Epto overflow--hidden__09f24__7Mh-7" aria-label="3.5 star rating" role="img"><img class=" offscreen__09f24__1fcyM" src="https://s3-media0.fl.yelpcdn.com/assets/public/stars_v2.yji-52d3d7a328db670d4402843cbddeed89.png" width="132" height="560" alt=""></div>
    
tags(type): 
    <span class=" css-epvm6 display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><a href="/search?cflt=newamerican&amp;find_loc=Manchester%2C+MO" class="css-1634hvh" target="" name="" rel="" role="link"><button class="badge__09f24__xWRsZ color--gray-extra-light__09f24__1ZCSL size--large__09f24__2JW1p badge--interactive__09f24__bhLsk"><p class="text__09f24__2NHRu css-1hx6l2b">American (New)</p></button></a><a href="/search?cflt=italian&amp;find_loc=Manchester%2C+MO" class="css-1634hvh" target="" name="" rel="" role="link"><button class="badge__09f24__xWRsZ color--gray-extra-light__09f24__1ZCSL size--large__09f24__2JW1p badge--interactive__09f24__bhLsk"><p class="text__09f24__2NHRu css-1hx6l2b">Italian</p></button></a><a href="/search?cflt=sportsbars&amp;find_loc=Manchester%2C+MO" class="css-1634hvh" target="" name="" rel="" role="link"><button class="badge__09f24__xWRsZ color--gray-extra-light__09f24__1ZCSL size--large__09f24__2JW1p badge--interactive__09f24__bhLsk"><p class="text__09f24__2NHRu css-1hx6l2b">Sports Bars</p></button></a></span>
    
tags(price):
    <span class="priceRange__09f24__2GspP css-e81eai">$$</span>
    
tags(dining options):
    <div class=" border-color--default__09f24__3Epto" data-testid="TRUSTED_PROPERTY"><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-checkmark-v2 css-1768wcw"><svg width="16" height="16" class="icon_svg"><path d="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-1x0u7iy"><span class=" raw__09f24__3Azrj">Indoor &amp; Outdoor dining</span></p></span></div><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-checkmark-v2 css-1768wcw"><svg width="16" height="16" class="icon_svg"><path d="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-1x0u7iy"><span class=" raw__09f24__3Azrj">Delivery</span></p></span></div><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-checkmark-v2 css-1768wcw"><svg width="16" height="16" class="icon_svg"><path d="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-1x0u7iy"><span class=" raw__09f24__3Azrj">Takeout</span></p></span></div></div>
    
contains all of the above:
    <div class=" arrange-unit__09f24__eFC_S arrange-unit-fill__09f24__1bMmp border-color--default__09f24__3Epto"><div class=" border-color--default__09f24__3Epto"><div class=" border-color--default__09f24__3Epto"><div class=" border-color--default__09f24__3Epto"><div class=" businessName__09f24__3Ml2X display--inline-block__09f24__3SvIn border-color--default__09f24__3Epto" aria-hidden="false"><div class=" border-color--default__09f24__3Epto"><h4 class="css-1l5lt1i"><span class=" css-1pxmz4g">5<!-- -->.&nbsp;<a href="/biz/joey-bs-food-and-drink-ballwin" class="css-166la90" target="" name="Joey B’s Food &amp; Drink" rel="">Joey B’s Food &amp; Drink</a></span></h4></div></div></div><div class=" border-color--default__09f24__3Epto"><div class=" display--inline-block__09f24__3SvIn border-color--default__09f24__3Epto" aria-hidden="false"><div class=" border-color--default__09f24__3Epto"><div class=" attribute__09f24__1La1D display--inline-block__09f24__3SvIn margin-r1__09f24__3PebR border-color--default__09f24__3Epto"><span class=" display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><div class=" i-stars__09f24___sZu0 i-stars--regular-3-half__09f24__1ufzF border-color--default__09f24__3Epto overflow--hidden__09f24__7Mh-7" aria-label="3.5 star rating" role="img"><img class=" offscreen__09f24__1fcyM" src="https://s3-media0.fl.yelpcdn.com/assets/public/stars_v2.yji-52d3d7a328db670d4402843cbddeed89.png" width="132" height="560" alt=""></div></span></div><div class=" attribute__09f24__1La1D display--inline-block__09f24__3SvIn border-color--default__09f24__3Epto"><span class="reviewCount__09f24__3GsGY css-e81eai">270</span></div></div></div></div><div class=" border-color--default__09f24__3Epto"><div class=" priceCategory__09f24__Ylk7h iaPriceCategory__09f24__307ME display--inline-block__09f24__3SvIn margin-t1__09f24__3NkZw border-color--default__09f24__3Epto" aria-hidden="false"><div class=" border-color--default__09f24__3Epto"><div class=" border-color--default__09f24__3Epto"><p class="css-1j7sdmt"><span class=" css-epvm6 display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><a href="/search?cflt=newamerican&amp;find_loc=Manchester%2C+MO" class="css-1634hvh" target="" name="" rel="" role="link"><button class="badge__09f24__xWRsZ color--gray-extra-light__09f24__1ZCSL size--large__09f24__2JW1p badge--interactive__09f24__bhLsk"><p class="text__09f24__2NHRu css-1hx6l2b">American (New)</p></button></a><a href="/search?cflt=italian&amp;find_loc=Manchester%2C+MO" class="css-1634hvh" target="" name="" rel="" role="link"><button class="badge__09f24__xWRsZ color--gray-extra-light__09f24__1ZCSL size--large__09f24__2JW1p badge--interactive__09f24__bhLsk"><p class="text__09f24__2NHRu css-1hx6l2b">Italian</p></button></a><a href="/search?cflt=sportsbars&amp;find_loc=Manchester%2C+MO" class="css-1634hvh" target="" name="" rel="" role="link"><button class="badge__09f24__xWRsZ color--gray-extra-light__09f24__1ZCSL size--large__09f24__2JW1p badge--interactive__09f24__bhLsk"><p class="text__09f24__2NHRu css-1hx6l2b">Sports Bars</p></button></a></span><span class=" display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><span class="priceRange__09f24__2GspP css-e81eai">$$</span></span><span class=" css-e81eai"></span></p></div></div></div></div></div></div><div class=" border-color--default__09f24__3Epto"><div class=" display--inline-block__09f24__3SvIn margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto" aria-hidden="false"><div class=" border-color--default__09f24__3Epto"><ul class=" undefined list__09f24__36n6_"><li class=" border-color--default__09f24__3Epto"><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-clock-v2 css-1g6mfbx"><svg width="16" height="16" class="icon_svg"><path d="M8 14.75A6.75 6.75 0 1114.75 8 6.757 6.757 0 018 14.75zm0-12A5.25 5.25 0 1013.25 8 5.256 5.256 0 008 2.75z"></path><path d="M10 8.919H8a.75.75 0 01-.75-.75v-3a.75.75 0 011.5 0v2.25H10a.75.75 0 110 1.5z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-e81eai"><span class=" raw__09f24__3Azrj">Waitlist is closed</span></p></span></div></li></ul></div></div></div><div class=" border-color--default__09f24__3Epto"><div class=" display--inline-block__09f24__3SvIn margin-t1__09f24__3NkZw border-color--default__09f24__3Epto" aria-hidden="false"><div class=" border-color--default__09f24__3Epto"><div class=" snippetTag__09f24__2G8wN arrange__09f24__1Vghi border-color--default__09f24__3Epto"><div class=" arrange-unit__09f24__eFC_S border-color--default__09f24__3Epto"><div class=" icon__09f24__3o_Ex css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-speech-v2 css-1qkiuob"><svg width="16" height="16" class="icon_svg"><path d="M5 14.309a.749.749 0 01-.75-.75v-2.45a3.768 3.768 0 01-3-3.667V5.44A3.754 3.754 0 015 1.69h6a3.754 3.754 0 013.75 3.75v2A3.755 3.755 0 0111 11.19H8.924l-3.437 2.938a.75.75 0 01-.487.18zM5 3.191a2.253 2.253 0 00-2.25 2.25v2a2.259 2.259 0 002.215 2.25.792.792 0 01.785.75v1.49l2.41-2.06a.749.749 0 01.487-.18H11a2.253 2.253 0 002.25-2.25v-2A2.253 2.253 0 0011 3.19H5z"></path></svg></span></div></div><div class=" arrange-unit__09f24__eFC_S arrange-unit-fill__09f24__1bMmp border-color--default__09f24__3Epto"><p class=" css-e81eai">“Arrived here and was seated promptly for an afternoon lunch. This place has an awesome, massive patio. They have ample TVs to watch any sports. The menu is…”<span class=" css-7onyyv">&nbsp;<a href="/biz/joey-bs-food-and-drink-ballwin?hrid=w4ASRZju7UYq7d1mD4LN-g" class="css-ac8spe" target="" name="" rel="">more</a></span></p></div></div></div></div></div><div class=" border-color--default__09f24__3Epto"></div><div class=" margin-t1-5__09f24__3aq3Q border-color--default__09f24__3Epto"><div class=" css-dojgf2 margin-b2__09f24__3lrcZ border-color--default__09f24__3Epto background-color--gray-light__09f24__2XmF7"></div><div class=" css-4kbcid arrange__09f24__1Vghi gutter-1__09f24__KR8eI vertical-align-middle__09f24__38J3B border-color--default__09f24__3Epto"><div class=" arrange-unit__09f24__eFC_S arrange-unit-fill__09f24__1bMmp border-color--default__09f24__3Epto"><div class=" border-color--default__09f24__3Epto"><div class=" display--inline-block__09f24__3SvIn border-color--default__09f24__3Epto" aria-hidden="false"><div class=" border-color--default__09f24__3Epto"><ul class=" undefined list__09f24__36n6_"><li class=" border-color--default__09f24__3Epto"><div class=" border-color--default__09f24__3Epto" data-testid="TRUSTED_PROPERTY"><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-checkmark-v2 css-1768wcw"><svg width="16" height="16" class="icon_svg"><path d="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-1x0u7iy"><span class=" raw__09f24__3Azrj">Indoor &amp; Outdoor dining</span></p></span></div><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-checkmark-v2 css-1768wcw"><svg width="16" height="16" class="icon_svg"><path d="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-1x0u7iy"><span class=" raw__09f24__3Azrj">Delivery</span></p></span></div><div class=" tag__09f24__Ox_hs css-1v994a0 margin-t0-5__09f24__75EiB border-color--default__09f24__3Epto"><div class=" icon__09f24__3XjHU css-1v994a0 border-color--default__09f24__3Epto"><span aria-hidden="true" class="icon--16-checkmark-v2 css-1768wcw"><svg width="16" height="16" class="icon_svg"><path d="M6.308 11.763a.748.748 0 01-.53-.22l-2.641-2.64a.75.75 0 011.06-1.061l2.11 2.11 5.496-5.495a.75.75 0 111.06 1.06l-6.025 6.026a.748.748 0 01-.53.22z"></path></svg></span></div><span class=" label__09f24__2sk0T display--inline__09f24__nqZ_W border-color--default__09f24__3Epto"><p class="tagText__09f24__1TL3S iaTagText__09f24__6z9F_ css-1x0u7iy"><span class=" raw__09f24__3Azrj">Takeout</span></p></span></div></div></li></ul></div></div></div></div></div></div></div>

In [ ]:
all_quotes = []
url = 'http://quotes.toscrape.com/'
browser.visit(url)

for i in range(1,5):    
    soup = bs(browser.html, 'html.parser')
    all_divs = soup.find_all('div',class_='quote')
    for div in all_divs:
        try:
            quote_text = div.find('span',class_='text').text
            author = div.find('small', class_='author').text
            tags = div.find('div',class_='tags').find_all('a')
            tags_list=[]
            for tag in tags:
                tags_list.append(tag.text)
                
            quote_dict = {'quote':quote_text,
                         'author':author,
                         'tags':tags_list}
            #print(quote_dict)
            
            all_quotes.append(quote_dict)
            
        except AttributeError as e:
            print(e)
    browser.links.find_by_partial_text('Next').click()
df = pd.DataFrame(all_quotes)
df

In [121]:
list1 = ['1', '2']
list2 = ['3', '4']
list3 = list1 + list2
print(list3)

['1', '2', '3', '4']
